In [1]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.datasets import load_dd
from time import time

Example to test it with: 1808 samples

In [2]:
df = load_dd()
cox = CoxPHFitter()
df['democracy'] = df['democracy'] == 'Democracy'
cox.fit(df[['duration', 'observed', 'democracy']], duration_col='duration', event_col='observed')
cox.concordance_index_, len(df)

(0.5927071741643755, 1808)

In [3]:
# Event times
t = df['duration'].to_numpy()

# Event indicator
e = df['observed'].to_numpy().astype(bool)

# Risk prediction
p = -cox.predict_partial_hazard(df[['democracy']]).to_numpy()

My big brain custom implementation. Can be improved further, namely: 
- `(t_ > 0) & e[:, np.newaxis]` is calculated 3 times 
- `(t_ < 0) & e[np.newaxis])` is calculated 3 times

In [4]:
def c_index_improved():

    # Shape: [n, n], where n is the number of samples
    # t_[i,j] is positive if i lives more than j
    # t_[i,j] is negative if i lives less than j
    # t_[i,j] = 0 if i and j die at the same time
    t_ = t[np.newaxis] - t[:, np.newaxis]

    # Do the same for the risk scores
    p_ = p[np.newaxis] - p[:, np.newaxis]

    # Concordant pairs are all of the pairs (i,j) where 
    # t[i]>t[j] and p[i]>p[j] and j is uncensored
    # OR
    # t[i]<t[j] and p[i]<p[j] and i is uncensored
    concordant = np.stack((((t_ > 0) & (p_ > 0) & e[:, np.newaxis]) | ((t_ < 0) & (p_ < 0) & e[np.newaxis])).nonzero(), axis=1)
    
    # Apply the same logic for discordant pairs
    discordant = np.stack((((t_ > 0) & (p_ < 0) & e[:, np.newaxis]) | ((t_ < 0) & (p_ > 0) & e[np.newaxis])).nonzero(), axis=1)
    
    # Apply the same logic for tied pairs
    tied       = np.stack((((t_ > 0) & (p_ == 0) & e[:, np.newaxis]) | ((t_ < 0) & (p_ == 0) & e[np.newaxis])).nonzero(), axis=1)

    c_index = (len(concordant) + .5 * len(tied)) / (len(concordant) + len(tied) + len(discordant))
    return c_index, concordant, discordant

start = time()
my_c, my_conc, my_disc = c_index_improved()
print(f'Time: {time() - start :.4f}, C index = {my_c:.4f}')

Time: 0.0911, C index = 0.5948


This is the original implementation copy-pasted from github:

In [5]:
def _c_index_concordant():
    
    event_times = t # I set this to my event times
    event_observed = e # I set this to my event indicators
    y_pred = p # I set to my risk estimation
    # Everything else is copy-paste
    
    concordant = 0
    discordant = 0
    tied = 0
    concordant_pairs = []
    discordant_pairs = []
    for i in range(len(event_times)):
        for j in range(len(event_times)):
            if i == j or (event_observed[i] == 0 and event_observed[j] == 0):
                continue
            if event_observed[i] == 1 and event_observed[j] == 1:
                if event_times[i] < event_times[j]:
                    if y_pred[i] < y_pred[j]:
                        concordant += 1
                        concordant_pairs.append((i, j))
                    elif y_pred[i] > y_pred[j]:
                        discordant += 1
                        discordant_pairs.append((i, j))
                    else:
                        tied += 1
                elif event_times[i] > event_times[j]:
                    if y_pred[i] > y_pred[j]:
                        concordant += 1
                        concordant_pairs.append((i, j))
                    elif y_pred[i] < y_pred[j]:
                        discordant += 1
                        discordant_pairs.append((i, j))
                    else:
                        tied += 1
            elif event_observed[i] == 1 and event_observed[j] == 0:
                if event_times[i] < event_times[j]:
                    if y_pred[i] < y_pred[j]:
                        concordant += 1
                        concordant_pairs.append((i, j))
                    elif y_pred[i] > y_pred[j]:
                        discordant += 1
                        discordant_pairs.append((i, j))
                    elif y_pred[i] == y_pred[j]:
                        tied += 1
                elif event_times[i] > event_times[j]:
                    # we dont know if this is concordant or discordant
                    continue
            elif event_observed[i] == 0 and event_observed[j] == 1:
                if event_times[j] < event_times[i]:
                    if y_pred[j] < y_pred[i]:
                        concordant += 1
                        concordant_pairs.append((i, j))
                    elif y_pred[j] > y_pred[i]:
                        discordant += 1
                        discordant_pairs.append((i, j))
                    elif y_pred[i] == y_pred[j]:
                        tied += 1
                elif event_times[j] > event_times[i]:
                    # we dont know if this is concordant or discordant
                    continue

    return (
        (concordant + 0.5 * tied) / (concordant + tied + discordant),
        concordant_pairs,
        discordant_pairs,
    )

start = time()
c, conc, disc = _c_index_concordant()
print(f'Time: {time() - start :.4f}, C index = {c:.4f}')

Time: 10.6841, C index = 0.5948


The results are exactly the same and my implementation is literally 100 times faster :)

In [6]:
print(np.array_equal(my_conc, conc))
print(np.array_equal(my_disc, disc))

True
True
